In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

Importer des données

In [ ]:
diabete = pd.read_csv("../input/pyms-diabete/diabete.csv")

In [ ]:
diabete.head().T

In [ ]:
diabete.describe()

counter les nombres de diabete

In [ ]:
diabete.diabete.value_counts()

In [ ]:
fig = sns.FacetGrid(diabete, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "n_pregnant", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(diabete, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "glucose", shade=True)
fig.add_legend()

In [ ]:
sns.set(style='ticks')
plt.figure(figsize=(20,20))
sns.pairplot(diabete, hue='diabete')

## Trouver des données inutiles

Avec des graphiques ci-dessus, on voie qu'il existe des données qui sont équales à 0, il sont pas possibles pour les caractéristiques physiques (Insulin = 0 c'est possible)

Tension: 

In [ ]:
print(diabete[diabete.tension == 0].shape[0])
print(diabete[diabete.tension == 0].index.tolist()) # tension n'est pas possible d'être equal à0
print(diabete[diabete.tension == 0].groupby('diabete')['age'].count())

Glucose:

In [ ]:
print(diabete[diabete.glucose == 0].shape[0])
print(diabete[diabete.glucose == 0].index.tolist()) # Glucose n'est pas possible d'être equal à0
print(diabete[diabete.glucose == 0].groupby('diabete')['age'].count())

thickness

In [ ]:
print(diabete[diabete.thickness == 0].shape[0])
print(diabete[diabete.thickness == 0].index.tolist())
print(diabete[diabete.thickness == 0].groupby('diabete')['age'].count())

Pour thickness, il y a trop des données qui sont équales à 0

BMI

In [ ]:
print(diabete[diabete.bmi == 0].shape[0])
print(diabete[diabete.bmi == 0].index.tolist())
print(diabete[diabete.bmi == 0].groupby('diabete')['age'].count())

On peut supprimer des données inutiles:

In [ ]:
data = diabete[(diabete.tension != 0) & (diabete.bmi != 0) & (diabete.glucose!= 0)]

## Visualisations

1. n_pregnant

In [ ]:
fig = sns.FacetGrid(data, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "n_pregnant", shade=True)
fig.add_legend()

Les 2 courbes sont confondues

2. Tension

In [ ]:
fig = sns.FacetGrid(data, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "tension", shade=True)
fig.add_legend()

3. Glucose

In [ ]:
fig = sns.FacetGrid(data, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "glucose", shade=True)
fig.add_legend()

4. Thickness

In [ ]:
fig = sns.FacetGrid(data, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "thickness", shade=True)
fig.add_legend()

5. bmi

In [ ]:
fig = sns.FacetGrid(data, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "bmi", shade=True)
fig.add_legend()

6. Insulin

In [ ]:
fig = sns.FacetGrid(data, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "insulin", shade=True)
fig.add_legend()

## Comme indiqué par les graphiques ci-dessus, on peut voir que le glucose est corrélé avec le diabète

## Mechine Learning

Séparer des donnée par le set train et le set test

In [ ]:
X = data.drop(['diabete'], axis=1) # sauf cible
y = data.diabete

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_lr))

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

Pour faciliter la visualisation

In [ ]:
dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

La distribution a une partie de confusion

## Forêt aléatoire

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

On sélectionne le meilleur estimateur

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

Importance des cartactéristiques

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

## On voie clairement que comme prévu le glucose est corrélé avec le diabète

## Xgboost

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

## Les méthodes de classification

1. Linear SVC

In [ ]:
from sklearn.svm import SVC, LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
Y_pred = linear_svc.predict(X_test)
print(accuracy_score(y_test,Y_pred))

2. SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(X_train, y_train)
Y_pred = sgd.predict(X_test)
print(accuracy_score(y_test,Y_pred))

In [ ]:
from sklearn.kernel_approximation import RBFSampler

rbf = RBFSampler(random_state=1)
X_features=rbf.fit_transform(X_test)
# clf = SGDClassifier()
sgd.fit(X_features,y_test)
print(sgd.score(X_features,y_test))

KNeighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
print(accuracy_score(y_test,Y_pred))